<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train6_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:

train6 = pd.read_csv(r'/content/train_data6.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [4]:

train6['Target'] = train6['Target'].replace(['Others'],'Invalid')


In [5]:
train6['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [6]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train6_Xtfidf = tfidf_vect.fit_transform(train6['Sentence'])


In [9]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [10]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [11]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [12]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [13]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [14]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [15]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train6_Xtfidf,train6['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6980088495575221
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       121
      Conclusion       0.84      0.56      0.68        48
           Facts       0.75      0.92      0.83       494
         Invalid       0.53      0.46      0.50       127
           Issue       0.59      0.45      0.51        49
Rule/Law/Holding       0.66      0.29      0.40        65

        accuracy                           0.70       904
       macro avg       0.65      0.52      0.56       904
    weighted avg       0.68      0.70      0.68       904



In [16]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [17]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3]
[0.9949578968488035, 0.9967988043461921, 0.9999088559265313, 0.9989651512370008, 0.9963649927629924, 0.9942612760768907, 0.9822438412774602, 0.9994536717906463, 0.9981741773290917, 0.9974417407605776, 0.9828036109148779, 0.9996190849175178, 0.9916713625039099, 0.9852637551241183, 0.9917464516526389, 0.9961745481219794, 0.9960039558129243, 0.9839332125833189, 0.999295384503781, 0.9997502063431699, 0.9872353819287399, 0.9964272883102572, 0.9897349498513035, 0.9866557675935902, 0.9876920498989094, 0.9955487935134482, 0.987261459940335, 0.9986056603379874, 0.9989531035673654, 0.983795521438327, 0.9942126888848498, 0.999562727832008, 0.9907605304243169, 0.9896143851275753, 0.9898505019831847, 0.990823148771635, 0.9991452857683154, 0.9992510960592955, 0.9985536078387475, 0.9951420989650319]
[0, 4, 5, 11, 12, 17, 18, 23, 24, 29, 30, 31, 33, 36, 37, 39, 40, 41, 45, 48, 49, 5

In [18]:
unlabel_1=unlabel_1.loc[[0, 4, 5, 11, 12, 17, 18, 23, 24, 29, 30, 31, 33, 36, 37, 39, 40, 41, 45, 48, 49, 50, 51, 52, 55, 56, 59, 60, 62, 67, 70, 76, 78, 79, 84, 87, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts


In [19]:
frame_1 = [train6,unlabel_1]
train6_1 = pd.concat(frame_1)
len(train6_1)

3516

In [20]:
x_train_1 = tfidf_vect.transform(train6_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train6_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6846590909090909
                  precision    recall  f1-score   support

        Analysis       0.53      0.49      0.51       100
      Conclusion       0.76      0.59      0.67        37
           Facts       0.75      0.90      0.81       355
         Invalid       0.55      0.51      0.53       109
           Issue       0.71      0.50      0.59        44
Rule/Law/Holding       0.64      0.24      0.35        59

        accuracy                           0.68       704
       macro avg       0.66      0.54      0.58       704
    weighted avg       0.67      0.68      0.67       704



In [21]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9964383808249301, 0.9811204129498791, 0.9995013204155335, 0.9887023426960472, 0.9923561956637864, 0.9976051364450116, 0.9999368643970972, 0.9867421033463073, 0.9994301470588987, 0.9883980816500483, 0.9985455536702567, 0.9941749919461976, 0.9960174243984942, 0.9996738725453559, 0.9999430357146807, 0.999200813499469, 0.9999362182555708, 0.9995097064517768, 0.998406375350513, 0.9964444237622561, 0.9994735517956517, 0.9995219980877118, 0.9998622298160247, 0.9991362399305095, 0.9954515218478631, 0.9971156715417318, 0.9961982915166145, 0.9917384631520456, 0.9994991730536236, 0.9959792190367058, 0.9906822783942009, 0.9994614531102467, 0.9994582688427077, 0.982984826903718, 0.9884163506195964]
[0, 1, 2, 4, 6, 7, 12, 13, 20, 21, 29, 31, 32, 36, 37, 39, 42, 49, 54, 55, 59, 63, 64, 69, 70, 71, 75, 77, 83, 85, 86, 88, 91, 95, 99]
35
35
35


In [22]:
unlabel_2 = unlabel_2.loc[[0, 1, 2, 4, 6, 7, 12, 13, 20, 21, 29, 31, 32, 36, 37, 39, 42, 49, 54, 55, 59, 63, 64, 69, 70, 71, 75, 77, 83, 85, 86, 88, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts


In [23]:
frame_2 = [train6_1,unlabel_2]
train6_2 = pd.concat(frame_2)
len(train6_2)

3551

In [24]:
x_train_2 = tfidf_vect.transform(train6_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train6_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6579770594369134
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       145
      Conclusion       0.81      0.45      0.58        47
           Facts       0.73      0.87      0.80       496
         Invalid       0.51      0.48      0.50       159
           Issue       0.63      0.43      0.51        56
Rule/Law/Holding       0.56      0.32      0.41        56

        accuracy                           0.66       959
       macro avg       0.62      0.49      0.54       959
    weighted avg       0.65      0.66      0.64       959



In [25]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9932200824618055, 0.9934680592291839, 0.9965858538197716, 0.9930819604364524, 0.9990264369309222, 0.9993680637763792, 0.9995539335788094, 0.9849677867173552, 0.9847362773708898, 0.9988400303852764, 0.993109768863758, 0.9858610976748065, 0.9997415589307295, 0.9849887381031026, 0.9936519060382604, 0.9986403443225369, 0.9816597038561551, 0.9994223793207797, 0.9951568481847691, 0.992580251992457, 0.9958029655072997, 0.9984240646919966, 0.9910118688829856, 0.9864208786159961, 0.9803941921708629, 0.996768176135473, 0.9921237566360309, 0.9982657166862641, 0.9958147357497219, 0.9957180383284883, 0.9997845764355641, 0.9922672366413116, 0.9994064215646029, 0.9984954291493723, 0.9997630460337931, 0.9979070506814275, 0.9919379940898565, 0.9995339212360025, 0.9985676391398656, 0.9853075416087196, 0.9980326266028313, 0.9977262997067048, 0.9970662979033958, 0.9970549

In [26]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 6, 7, 8, 12, 15, 17, 20, 23, 26, 27, 28, 29, 30, 32, 35, 36, 40, 43, 46, 47, 48, 51, 52, 53, 54, 56, 57, 63, 64, 65, 67, 68, 70, 71, 72, 75, 78, 79, 89, 96, 97],:]
frame_3 = [train6_2,unlabel_3]
train6_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train6_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train6_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6828929068150209
                  precision    recall  f1-score   support

        Analysis       0.42      0.38      0.40        84
      Conclusion       0.76      0.52      0.62        42
           Facts       0.76      0.89      0.82       383
         Invalid       0.54      0.44      0.48       128
           Issue       0.82      0.56      0.67        41
Rule/Law/Holding       0.50      0.39      0.44        41

        accuracy                           0.68       719
       macro avg       0.63      0.53      0.57       719
    weighted avg       0.67      0.68      0.67       719



In [27]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 5, 2, 2]
[0.9990947548995225, 0.9997028135648546, 0.9961545531624493, 0.9928001769409697, 0.9999569815815641, 0.9935218621415014, 0.99073089146092, 0.99841476121593, 0.9940648916018531, 0.9996958278520556, 0.9969825246782307, 0.9972819838383862, 0.9986406897598675, 0.9964232193045192, 0.9998322650206912, 0.9965676414243068, 0.9968181014966183, 0.9998487489513553, 0.9984081813114735, 0.9992694088850649, 0.9969319371406168, 0.9833320454716957, 0.9995141909086005, 0.9936702642701452, 0.9977570352196058, 0.9978508009755275, 0.9966649189157653, 0.9921840890669339, 0.9910727914361008, 0.9842824276934221, 0.9977631394794054, 0.9975204583976766, 0.9964746865264608, 0.9950195770200252, 0.999973478841108, 0.9934823943565324, 0.9911716821377609, 0.989798498775981, 0.9943085443467198, 0.9820604775325344, 0.9990639746879612, 0.9899329238666961, 0.9986582632938156, 0.9921637911

In [28]:
unlabel_4=unlabel_4.loc[[2, 4, 7, 8, 10, 11, 12, 15, 16, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 49, 50, 56, 58, 60, 61, 64, 67, 69, 70, 71, 75, 80, 82, 85, 87, 88, 93, 94, 97, 99],:]

In [29]:
frame_4 = [train6_3,unlabel_4]
train6_4 = pd.concat(frame_4)
len(train6_4)
x_train_4 = tfidf_vect.transform(train6_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train6_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6775178026449644
                  precision    recall  f1-score   support

        Analysis       0.45      0.46      0.46       133
      Conclusion       0.64      0.58      0.61        48
           Facts       0.79      0.87      0.82       530
         Invalid       0.49      0.45      0.47       148
           Issue       0.59      0.41      0.48        58
Rule/Law/Holding       0.64      0.41      0.50        66

        accuracy                           0.68       983
       macro avg       0.60      0.53      0.56       983
    weighted avg       0.67      0.68      0.67       983



In [30]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 3, 0]
[0.9993112520218093, 0.988110267362586, 0.9901976775188188, 0.9947829788467097, 0.9961940612728999, 0.9966618870590962, 0.9852129852673158, 0.9969593526981917, 0.9994725439059291, 0.9995240313569178, 0.9943804489631026, 0.9913295137138476, 0.9825200313192219, 0.9997475709410872, 0.9941863144544281, 0.9993723769215033, 0.9986461101622153, 0.9890676746882162, 0.9881831383967655, 0.9989306277150009, 0.9973146724133729, 0.9983189521593123, 0.9890408714866699, 0.991514337201505, 0.9955756682033463, 0.9843482203342616, 0.984748683528342, 0.981137547535609, 0.9998924708609902, 0.9989669901587355, 0.9951312146886531, 0.9954048415817592, 0.9983512258336521, 0.9985943019564207, 0.9869497465445546, 0.9944391187160051, 0.9901072374104813, 0.9921479005789428, 0.9913243880757237, 0.9884249012910311]
[3, 4, 7, 12, 18, 19, 20, 22, 23, 26, 27, 32, 35, 36, 46, 52, 53, 56, 59, 60, 63, 

In [31]:
unlabel_5 = unlabel_5.loc[[3, 4, 7, 12, 18, 19, 20, 22, 23, 26, 27, 32, 35, 36, 46, 52, 53, 56, 59, 60, 63, 65, 66, 67, 71, 73, 75, 78, 82, 86, 91, 93, 94, 95, 100, 103, 104, 106, 107, 110],:]

In [32]:
frame_5 = [train6_4,unlabel_5]
train6_5 = pd.concat(frame_5)
len(train6_5)
x_train_5 = tfidf_vect.transform(train6_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train6_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6881287726358148
                  precision    recall  f1-score   support

        Analysis       0.50      0.44      0.46       142
      Conclusion       0.69      0.56      0.62        55
           Facts       0.80      0.86      0.83       533
         Invalid       0.45      0.51      0.47       130
           Issue       0.68      0.60      0.64        57
Rule/Law/Holding       0.61      0.40      0.48        77

        accuracy                           0.69       994
       macro avg       0.62      0.56      0.58       994
    weighted avg       0.68      0.69      0.68       994



In [33]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6737044145873321
